In [1]:
import numpy as np
from scipy.optimize import minimize


class LogisticRegression(object):
    
    default_tol = 1e-10  # solver tol
    default_penalty = 1.0  # coef of the regularization term
    _exp_lim = 40  # exp overflow range

    def __init__(self, tol=default_tol,
                 standardize=True,
                 regularization=None,
                 penalty=default_penalty):
        self.tol = tol
        self.standardize = standardize
        self.penalty = penalty
        self.regularization = regularization
        
    def fit(self, X, y):
        self.n_params = len(X[0]) + 1  # include intercept
        X_train = self.transform(X, fit=True)
        y_train = self.convert_y(y, how="to_binary")
        beta_hat = minimize(self.loss,
                            x0=np.zeros(self.n_params),
                            method='BFGS',
                            tol=self.tol,
                            args=(X_train, y_train),
                           )
        self.beta = beta_hat.x
    
    def predict(self, X):
        X_test = self.transform(X, fit=False)
        Xbeta = (X_test * self.beta).sum(axis=1)
        p = np.fromiter((1. / (1 + np.exp(-xb)) if xb < self._exp_lim and xb > - self._exp_lim
                         else 1 if xb >= self._exp_lim
                         else 0 for xb in Xbeta), dtype=float)
        y_test = (p >= 0.5).astype(int)
        return self.convert_y(y_test, how="from_binary")
        
    def loss(self, beta, X_train, y_train):
        beta = np.array(beta)
        Xbeta = (X_train * beta).sum(axis=1)
        loglikelihood_p1 = (Xbeta * y_train).sum()
        # loglikelihood_p2 = np.log(1 + np.exp(Xbeta)).sum()
        # may need to approx to avoid overflow
        loglikelihood_p2 = np.sum(np.fromiter((np.log(1 + np.exp(xb)) if xb < self._exp_lim and xb > - self._exp_lim / 2
                                              else xb if xb >= self._exp_lim
                                              else np.exp(xb)
                                              for xb in Xbeta), dtype=float))
        loss = - loglikelihood_p1 - loglikelihood_p2
        if self.regularization == 'l1':
            loss += self.penalty * np.sum(np.abs(beta))
        elif self.regularization == 'l2':
            loss += self.penalty * np.sum(beta ** 2)
        return loss
    
    def transform(self, X, fit=True):
        X = np.array(X)
        if self.standardize:
            if fit:
                self.X_mean = X.mean(axis=0)
                self.X_std = X.std(axis=0)
                if 0. in self.X_std:
                    raise ZeroDivisionError('Please remove the column with identical values')
            X = (X - self.X_mean) / self.X_std
        return np.hstack([np.ones(len(X)).reshape(len(X), 1), X])
        
        
    def convert_y(self, y, how='to_binary'):
        if how == 'to_binary':
            self.levels = list(set(y))
            return np.array([0 if yi == self.levels[0] else 1 for yi in y])
        elif how == 'from_binary':
            return np.array([self.levels[yi] for yi in y])
        else:
            raise ValueError('argument "how" received an invalid string')

In [2]:
xys = [[[-1, 1, 1],  'a'],
       [[4, 4, 0],   'b'],
       [[2, -1, 2],  'a'],
       [[4, 6, 3],   'b'],
       [[0, 1, 5],   'a'],
       [[5, 7, 4],   'b'],
       [[0, 0, 6],   'a'],
       [[6, 6, 7],   'b'],
       [[-1, -1, 9], 'a'],
       [[7, 4, 8],   'b']]

In [3]:
a = LogisticRegression(regularization='l2')
a.fit(*list(zip(*xys)))
a.beta

array([ 5.61770794, -4.26112463, -4.2781618 , -0.60273384])

In [4]:
a.predict([[8, 5, 0], [0, -1, 10]])

array(['b', 'a'], dtype='<U1')